# Spotify Temporal and Genre Analysis

## Overview
This notebook provides additional insights into the Spotify dataset by focusing on temporal patterns and genre-specific characteristics. We'll explore how song features and popularity have evolved over time and how they vary across different genres.

## Analysis Goals:
1. **Temporal Patterns**
   - How have song characteristics changed over time?
   - Are there seasonal patterns in song popularity?
   - Which genres have gained or lost popularity?

2. **Genre Deep Dive**
   - What makes songs popular in different genres?
   - How do audio features vary across genres?
   - Which genres have the most consistent popularity?

3. **Feature Interactions**
   - How do combinations of features affect popularity?
   - Are there genre-specific feature patterns?
   - What role does release timing play?

In [12]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import calendar

# Set style for better visualizations
plt.style.use('seaborn')
sns.set_palette('viridis')
plt.rcParams['figure.figsize'] = [12, 6]

ModuleNotFoundError: No module named 'pandas'

In [ ]:
# Load and prepare data
df = pd.read_csv('../Untitled/Resources/spotify_songs.csv')

# Convert release date to datetime
df['release_date'] = pd.to_datetime(df['track_album_release_date'])
df['release_year'] = df['release_date'].dt.year
df['release_month'] = df['release_date'].dt.month
df['release_season'] = pd.cut(df['release_month'], 
                             bins=[0, 3, 6, 9, 12], 
                             labels=['Winter', 'Spring', 'Summer', 'Fall'])

## 1. Temporal Analysis

In [ ]:
# Analyze popularity trends over time
yearly_popularity = df.groupby('release_year')['track_popularity'].agg(['mean', 'std']).reset_index()

plt.figure(figsize=(15, 6))
plt.fill_between(yearly_popularity['release_year'],
                 yearly_popularity['mean'] - yearly_popularity['std'],
                 yearly_popularity['mean'] + yearly_popularity['std'],
                 alpha=0.2)
plt.plot(yearly_popularity['release_year'], yearly_popularity['mean'], linewidth=2)
plt.title('Song Popularity Trends Over Time')
plt.xlabel('Release Year')
plt.ylabel('Average Popularity Score')
plt.show()

# Seasonal analysis
seasonal_popularity = df.groupby('release_season')['track_popularity'].mean().reset_index()
plt.figure()
sns.barplot(data=seasonal_popularity, x='release_season', y='track_popularity')
plt.title('Average Song Popularity by Season')
plt.show()

## 2. Genre Analysis

In [ ]:
# Analyze genre characteristics
genre_stats = df.groupby('playlist_genre').agg({
    'track_popularity': ['mean', 'std'],
    'danceability': 'mean',
    'energy': 'mean',
    'valence': 'mean'
}).round(2)

# Plot genre characteristics
plt.figure(figsize=(15, 8))
genre_features = ['danceability', 'energy', 'valence']
for i, feature in enumerate(genre_features):
    plt.subplot(1, 3, i+1)
    sns.boxplot(data=df, x='playlist_genre', y=feature)
    plt.xticks(rotation=45)
    plt.title(f'{feature.capitalize()} by Genre')
plt.tight_layout()
plt.show()

## 3. Feature Interaction Analysis

In [ ]:
# Create feature combinations
df['energy_valence'] = df['energy'] * df['valence']
df['dance_energy_ratio'] = df['danceability'] / df['energy']

# Analyze feature interactions by genre
plt.figure(figsize=(15, 6))
sns.scatterplot(data=df, x='energy_valence', y='track_popularity', 
                hue='playlist_genre', alpha=0.5)
plt.title('Popularity vs Energy-Valence Interaction by Genre')
plt.show()

# Create popularity heatmap
pivot_table = df.pivot_table(values='track_popularity', 
                            index=pd.qcut(df['energy'], 5), 
                            columns=pd.qcut(df['danceability'], 5),
                            aggfunc='mean')

plt.figure(figsize=(10, 8))
sns.heatmap(pivot_table, annot=True, fmt='.0f', cmap='YlOrRd')
plt.title('Popularity Heatmap: Energy vs Danceability')
plt.xlabel('Danceability Quintiles')
plt.ylabel('Energy Quintiles')
plt.show()

## 4. Genre Evolution Analysis

In [ ]:
# Analyze how genres have evolved
genre_year_stats = df.groupby(['playlist_genre', 'release_year']).agg({
    'track_popularity': 'mean',
    'energy': 'mean',
    'danceability': 'mean'
}).reset_index()

# Plot genre evolution
plt.figure(figsize=(15, 6))
for genre in df['playlist_genre'].unique():
    genre_data = genre_year_stats[genre_year_stats['playlist_genre'] == genre]
    plt.plot(genre_data['release_year'], 
             genre_data['track_popularity'], 
             label=genre, 
             marker='o')

plt.title('Genre Popularity Evolution Over Time')
plt.xlabel('Release Year')
plt.ylabel('Average Popularity')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

## Key Findings

1. **Temporal Patterns**
   - Identify any seasonal trends in song releases and popularity
   - Note changes in audio features over time
   - Highlight years with notably high/low popularity scores

2. **Genre Insights**
   - Document characteristic features of successful songs in each genre
   - Note genres with highest average popularity
   - Identify unique feature patterns in different genres

3. **Feature Interactions**
   - Describe how combined features affect popularity
   - Note any genre-specific feature combinations
   - Highlight optimal feature ranges for popularity

4. **Evolution Trends**
   - Document how genres have evolved
   - Note emerging or declining trends
   - Identify successful adaptation patterns